In [10]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = glob.glob('../features/*.pkl')
for l in fl:
    print("'{}',".format(l))

'../features/vgg16_1_feat.pkl',
'../features/cnn_1_feat.pkl',
'../features/resnet_aug1_feat.pkl',
'../features/incept_aug1_feat.pkl',
'../features/other_feat.pkl',
'../features/incept_aug2_feat.pkl',
'../features/cnn_1_aug1_feat.pkl',
'../features/resnet_1_feat.pkl',
'../features/cnn_5_aug_rescale_preprocess_feat.pkl',
'../features/incept_aug4_feat.pkl',
'../features/resnet_aug4_feat.pkl',
'../features/cnn_4_aug_rescale_preprocess_feat.pkl',
'../features/cnn_3_aug1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_3_aug_rescale_preprocess_feat.pkl',
'../features/cnn_2_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_5_aug1_feat.pkl',
'../features/incept_1_feat.pkl',
'../features/resnet_aug3_feat.pkl',
'../features/incept_aug3_feat.pkl',
'../features/vgg_aug1_feat.pkl',
'../features/cnn_2_aug_rescale_preprocess_feat.pkl',
'../features/resnet_aug2_feat.pkl',
'../features/cnn_1_aug_rescale_preprocess_feat.pkl',


In [11]:
choose_fl = [
#'../features/cnn_1_feat.pkl',
#'../features/resnet_aug1_feat.pkl',
#'../features/incept_aug2_feat.pkl',
#'../features/cnn_3_aug1_feat.pkl',
#'../features/cnn_2_feat.pkl',
#'../features/cnn_4_aug1_feat_add_early.pkl',
#'../features/cnn_5_aug1_feat.pkl',
#'../features/incept_1_feat.pkl',
#'../features/incept_aug3_feat.pkl',
#'../features/vgg_aug1_feat.pkl',
'../features/vgg16_1_feat.pkl',
'../features/incept_aug1_feat.pkl',
'../features/other_feat.pkl',
'../features/cnn_1_aug1_feat.pkl',
'../features/resnet_1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/resnet_aug3_feat.pkl',
'../features/resnet_aug2_feat.pkl',
  
]

def load_x_y():
    train_x,test_x=[],[]
    for f in choose_fl:
        print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

../features/vgg16_1_feat.pkl
../features/incept_aug1_feat.pkl
../features/other_feat.pkl
../features/cnn_1_aug1_feat.pkl
../features/resnet_1_feat.pkl
../features/cnn_2_aug1_feat.pkl
../features/cnn_4_aug1_feat.pkl
../features/resnet_aug3_feat.pkl
../features/resnet_aug2_feat.pkl
(1604, 19) (8424, 19) (1604,)


In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 1.0
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=30,
                        verbose_eval=1009)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [13]:
cv_test(3,42,True)

[0]	train-logloss:0.639092	valid-logloss:0.643238
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[119]	train-logloss:0.07035	valid-logloss:0.163011

[0]	train-logloss:0.640488	valid-logloss:0.639944
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[81]	train-logloss:0.098805	valid-logloss:0.165829

[0]	train-logloss:0.640431	valid-logloss:0.64091
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[104]	train-logloss:0.082493	valid-logloss:0.163925

local average valid loss 0.166895443887 val loss std 0.00172757630585
         id  is_iceberg
0  5941774d    0.039962
1  4023181e    0.787347
2  b20200e4    0.0083

In [14]:
cv_test(5,42,True)

[0]	train-logloss:0.639003	valid-logloss:0.643103
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[159]	train-logloss:0.059304	valid-logloss:0.158494

[0]	train-logloss:0.640624	valid-logloss:0.642576
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[217]	train-logloss:0.047615	valid-logloss:0.124558

[0]	train-logloss:0.64019	valid-logloss:0.64249
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[128]	train-logloss:0.072304	valid-logloss:0.178184

[0]	train-logloss:0.640457	valid-logloss:0.640957
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid

In [15]:
cv_test(10,42,True)

[0]	train-logloss:0.640299	valid-logloss:0.644735
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[148]	train-logloss:0.066444	valid-logloss:0.176749

[0]	train-logloss:0.639776	valid-logloss:0.64181
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[185]	train-logloss:0.058884	valid-logloss:0.144427

[0]	train-logloss:0.640106	valid-logloss:0.641654
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[168]	train-logloss:0.0634	valid-logloss:0.132382

[0]	train-logloss:0.640288	valid-logloss:0.643088
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-